In [1]:
import pandas as pd
X = pd.read_csv('~/data/train_input_Z61KlZo.csv') 


/tmp/ipykernel_7912/31312324.py:2: DtypeWarning: Columns (16,17,29,30,31,126,128,129,132,133,135,138,371) have mixed types. Specify dtype option on import or set low_memory=False.
  X = pd.read_csv('~/data/train_input_Z61KlZo.csv')


In [5]:
Y = pd.read_csv('~/data/train_output_DzPxaPY.csv')

In [6]:
Y.head()

,ID,FREQ,CM,ANNEE_ASSURANCE,CHARGE
0,1,0.0,0.0,1.000000,0.0
1,2,0.0,0.0,1.000000,0.0
2,3,0.0,0.0,0.402740,0.0
3,4,0.0,0.0,0.246575,0.0
4,5,0.0,0.0,0.838356,0.0


In [7]:
y = Y["ANNEE_ASSURANCE"]

In [13]:
for col in X.select_dtypes(include=["object", "category"]).columns:
    X[col] = X[col].astype("category").cat.codes

In [15]:
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

# Séparer en jeu d'entraînement et de test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Créer et entraîner le modèle XGBoost
model = xgb.XGBRegressor(objective="reg:squarederror", n_estimators=100)
model.fit(X_train, y_train)

# Prédictions
y_pred = model.predict(X_test)

# Évaluation du modèle
mse = mean_squared_error(y_test, y_pred)
print(f"Mean Squared Error: {mse}")


Mean Squared Error: 0.0002556580986656133


In [12]:
X

,ID,ACTIVIT2,VOCATION,TYPERS,ANCIENNETE,ADOSS,CARACT1,CARACT2,CARACT3,INDEM1,...,NBJRR100_MMAX_A,NBJRR100_MSOM_A,RR_VOR_MM_A,RR_VOR_MMAX_A,RRAB_VOR_MM_A,RRAB_VOR_MMAX_A,ANNEE_ASSURANCE,ESPINSEE,AN_EXERC,ZONE
0,1,ACT1,VOC6,1,0,N,N,NaN,NaN,N,...,NaN,NaN,NaN,NaN,NaN,NaN,1.000000,NaN,ANNEE5,3
1,2,ACT1,VOC6,1,0,N,N,NaN,NaN,N,...,NaN,NaN,NaN,NaN,NaN,NaN,1.000000,NaN,ANNEE5,3
2,3,ACT1,VOC6,1,2,N,R,NaN,NaN,N,...,NaN,NaN,NaN,NaN,NaN,NaN,0.402740,NaN,ANNEE5,3
3,4,ACT1,VOC6,2,0,N,N,NaN,NaN,N,...,NaN,NaN,NaN,NaN,NaN,NaN,0.246575,NaN,ANNEE5,3
4,5,ACT1,VOC6,2,1,N,N,NaN,NaN,N,...,01. <= 0,01. <= 0,03. <= 102,03. <= 232,04. >= 25,02. <= 57,0.838356,NaN,ANNEE5,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
383605,383606,ACT5,VOC6,2,5,N,N,NaN,NaN,N,...,01. <= 0,01. <= 0,01. <= 63,01. <= 137,02. <= 19,01. <= 41,1.000000,ESP1,ANNEE6,95
383606,383607,ACT5,VOC6,2,11,N,N,NaN,NaN,N,...,01. <= 0,01. <= 0,01. <= 63,01. <= 137,02. <= 19,01. <= 41,1.000000,ESP1,ANNEE6,95
383607,383608,ACT1,VOC6,1,10,N,N,NaN,NaN,N,...,NaN,NaN,NaN,NaN,NaN,NaN,1.000000,ESP3,ANNEE6,95
383608,383609,ACT1,VOC6,2,0,N,N,NaN,NaN,N,...,01. <= 0,01. <= 0,02. <= 79,02. <= 176,02. <= 19,01. <= 41,1.000000,ESP1,ANNEE6,95


In [16]:
y_test

80050     0.923288
34634     0.200000
63168     1.000000
134612    0.328767
238463    0.896175
            ...   
89382     0.071038
125169    0.928767
257834    0.553425
317470    1.000000
130950    0.246575
Name: ANNEE_ASSURANCE, Length: 76722, dtype: float64